# JUMPING FIX

In [1]:
import geopandas as gp
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from shapely.geometry import Point
import nafot
import time
import random
from tqdm import tqdm, tqdm_notebook
from datetime import datetime as dt
from geopy.distance import distance
%matplotlib inline

In [2]:
loc_data = pd.read_csv(r'C:\Users\user\Documents\דרופבוקס\Data - cellular db\with stat area\sample2_100M_with_stat.csv')

# Remove records without stat area
loc_data.dropna(inplace=True)

In [3]:
im = np.random.choice(loc_data.imsi)
im = 52603953665
print (im)

52603953665


In [12]:
jumps_dfs = []
jumps = pd.DataFrame(columns=['total', 'jumps'])

for i in tqdm_notebook(range(1000)):
    im = np.random.choice(loc_data.imsi)

    loc = loc_data[loc_data.imsi == im].copy()
    loc['datetime'] = loc.apply(lambda row: dt.strptime('{} {}'.format(row.date_stamp, row.time_stamp), '%Y-%m-%d %X'),axis=1)

    # Sort by datetime
    loc.sort_values('datetime', inplace=True)

    # loc['point'] = loc.apply(lambda row: Point(row.latitude, row.longtitude), axis=1)
    # loc['prev_point'] = loc.point.shift()
    loc['prev_datetime'] = loc.datetime.shift()

    loc['prev_lat'] = loc.latitude.shift()
    loc['prev_lon'] = loc.longtitude.shift()


    # Remove first row
    loc.drop(loc.index[0], inplace=True)

    # Calculate distance and time difference
    # loc['dist'] = loc.apply(lambda row: row.point.distance(row.prev_point), axis=1)
    loc['dist'] = loc.apply(lambda row: distance((row.latitude,row.longtitude),(row.prev_lat,row.prev_lon)).meters, axis=1)
    loc['time_diff'] = loc.apply(lambda row: (row.datetime - row.prev_datetime).seconds, axis=1)
    loc['speed'] = loc.apply(lambda row: (row.dist/row.time_diff) if row.time_diff > 0 else 0 ,axis=1)

    loc = loc[['datetime', 'prev_datetime', 'latitude','longtitude', 'prev_lat', 'prev_lon', 'dist', 'time_diff', 'speed']]
    
    # Add the data to jumps df
    jumps.loc[i] = loc.shape[0], loc[loc.speed > 300].shape[0] 
    
    # Save df
    jumps_dfs.append(loc.copy())

jumps['jump_precent'] = jumps.jumps / jumps.total
jumps

,total,jumps,jump_precent
0,210155,229,0.00108967
1,31540,12,0.000380469
2,136075,103,0.000756936
3,37010,38,0.00102675
4,141715,575,0.00405744
5,20149,18,0.000893345
6,34711,178,0.00512806
7,39577,97,0.00245092
8,184954,738,0.00399018
9,27047,47,0.00173772


In [18]:
for i, df in enumerate(jumps_dfs):
    df['user'] = i

In [38]:
jumps.describe(percentiles=[0,.25,.5,.75,1])

,total,jumps,jump_precent
count,1000,1000,1000.0
unique,782,324,766.0
top,127101,0,0.0
freq,6,18,18.0


In [19]:
jumpd_df = pd.concat(jumps_dfs)

In [24]:
jump_df = jumpd_df[['user', 'datetime', 'prev_datetime', 'latitude', 'longtitude', 'prev_lat',
       'prev_lon', 'dist', 'time_diff', 'speed']].copy()

In [27]:
# jumps.to_csv('../data/jumps.csv', index=False)
# jump_df.to_csv('../data/jumps_df.csv', index=False)

In [4]:
jump_df = pd.read_csv('../data/jumps.csv')
jump_df.describe()

,total,jumps,jump_precent
count,1000.000000,1000.000000,1000.000000
mean,56073.336000,213.962000,0.003499
std,53588.897795,411.017951,0.003726
min,99.000000,0.000000,0.000000
25%,20096.000000,28.000000,0.001113
50%,42823.000000,92.000000,0.002436
75%,75852.750000,224.500000,0.004660
max,434690.000000,3978.000000,0.037332


In [6]:
jump_data = pd.read_csv(r'C:\Users\user\Documents\דרופבוקס\Data - cellular db\with stat area/jumps_df.csv')